In [1]:
import configparser
import json
import spotipy
import spotipy.util as util
import pandas as pd
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import bamboolib

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('../data/raw/spotify_creds.json') as f:
    spotify_creds = json.load(f)

In [3]:
client_id = spotify_creds['client_id']
client_secret = spotify_creds['client_secret']
username = spotify_creds['username']
scope = spotify_creds['scope']
redirect_uri = spotify_creds['redirect_url']

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [5]:
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

offset_index = 0

songs = sp.current_user_saved_tracks(offset=offset_index)

In [6]:
df_saved_tracks = pd.DataFrame()
track_list = ''
added_ts_list = []
artist_list = []
title_list = []
more_songs = True
offset_index = 0

while more_songs:
    songs = sp.current_user_saved_tracks(offset=offset_index)
    for song in songs['items']:
        #join track ids to a string for audio_features function
        track_list += song['track']['id'] +','
        #get the time when the song was added
        added_ts_list.append(song['added_at'])
        #get the title of the song
        title_list.append(song['track']['name'])
        #get all the artists in the song
        artists = song['track']['artists']
        artists_name = ''
        for artist in artists:
            artists_name += artist['name']  + ','
        artist_list.append(artists_name[:-1])
    #get the track features and append into a dataframe
    track_features = sp.audio_features(track_list[:-1])
    df_temp = pd.DataFrame(track_features)
    df_saved_tracks = df_saved_tracks.append(df_temp)
    track_list = ''
    if songs['next'] == None:
        # no more songs in playlist
        more_songs = False
    else:
        # get the next n songs
        offset_index += songs['limit']
#include timestamp added, title and artists of a song
df_saved_tracks['added_at'] = added_ts_list
df_saved_tracks['song_title'] = title_list
df_saved_tracks['artists'] = artist_list

In [8]:
df_saved_tracks['artists'].value_counts()

Jay Chou                            17
Justin Bieber                       14
Leehom Wang                         10
Lauv                                 9
Ed Sheeran                           8
                                    ..
Cheat Codes,Ashworth,Demi Lovato     1
Nicholas Lentz                       1
Alesso,BROHUG                        1
Chris Brown                          1
R3HAB,Noah Neiman,Miranda Glory      1
Name: artists, Length: 890, dtype: int64

In [15]:
a = df_saved_tracks[df_saved_tracks['song_title'] == 'The Spook']
a

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,added_at,song_title,artists
18,0.719,0.936,9,-3.343,1,0.0712,0.00515,0.0759,0.0723,0.294,...,audio_features,4yCo1eRj9ZWKcAcXxxztGS,spotify:track:4yCo1eRj9ZWKcAcXxxztGS,https://api.spotify.com/v1/tracks/4yCo1eRj9ZWK...,https://api.spotify.com/v1/audio-analysis/4yCo...,165938,4,2019-10-28T21:01:29Z,The Spook,"KSHMR,Basskillers,B3nte"


In [24]:
songs['items'][0]['track']['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/0Qnx1MPnHYt3jJCYrRFVwX'},
  'href': 'https://api.spotify.com/v1/artists/0Qnx1MPnHYt3jJCYrRFVwX',
  'id': '0Qnx1MPnHYt3jJCYrRFVwX',
  'name': 'Greyson Chance',
  'type': 'artist',
  'uri': 'spotify:artist:0Qnx1MPnHYt3jJCYrRFVwX'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/6cEuCEZu7PAE9ZSzLLc2oQ'},
  'href': 'https://api.spotify.com/v1/artists/6cEuCEZu7PAE9ZSzLLc2oQ',
  'id': '6cEuCEZu7PAE9ZSzLLc2oQ',
  'name': 'R3HAB',
  'type': 'artist',
  'uri': 'spotify:artist:6cEuCEZu7PAE9ZSzLLc2oQ'}]

In [67]:
df_saved_tracks = pd.DataFrame()
track_list = ''
added_ts_list = []
artist_list = []
title_list = []
popularity_list = []
album_cover_list = []
more_songs = True
offset_index = 0

while more_songs:
    songs = sp.current_user_saved_tracks(offset=offset_index)
    for song in songs['items']:
        #join track ids to a string for audio_features function
        track_list += song['track']['id'] +','
        #get the time when the song was added
        added_ts_list.append(song['added_at'])
        #get the title of the song
        title_list.append(song['track']['name'])
        #get popularity
        popularity_list.append(song['track']['popularity'])
        album_cover_list.append(song['track']['album']['images'][0]['url'])
        #get all the artists in the song
        artists = song['track']['artists']
        artists_name = ''
        for artist in artists:
            artists_name += artist['name']  + ','
        artist_list.append(artists_name[:-1])
    track_list = ''
    if songs['next'] == None:
        # no more songs in playlist
        more_songs = False
    else:
        # get the next n songs
        offset_index += songs['limit']
#include timestamp added, title and artists of a song
df_saved_tracks['song_title'] = title_list
df_saved_tracks['artists'] = artist_list
df_saved_tracks['date_added'] = added_ts_list
df_saved_tracks['album_cover'] = album_cover_list
df_saved_tracks['popularity'] = popularity_list

In [68]:
df_saved_tracks

,song_title,artists,date_added,album_cover,popularity
0,Nightmare,K-391,2022-02-26T19:55:18Z,https://i.scdn.co/image/ab67616d0000b2739f4a62...,45
1,If I Ain't Got You - Acoustic,John Adams,2022-02-21T05:09:51Z,https://i.scdn.co/image/ab67616d0000b273d2f483...,54
2,If I Ain't Got You,Alicia Keys,2022-02-21T05:09:25Z,https://i.scdn.co/image/ab67616d0000b27356ff19...,83
3,Lost At Sea (Illa Illa 2),"B.I,Bipolar Sunshine,Afgan",2022-02-20T22:04:25Z,https://i.scdn.co/image/ab67616d0000b2730c6c9a...,61
4,When I'm Gone (with Katy Perry) [VIP Mix],"Alesso,Katy Perry",2022-02-20T19:24:17Z,https://i.scdn.co/image/ab67616d0000b273b5b2dd...,67
...,...,...,...,...,...
1098,Lullaby,"R3HAB,Mike Williams",2018-04-06T15:56:41Z,https://i.scdn.co/image/ab67616d0000b27342ccfe...,64
1099,Hold Me - AmPm Remix,"R3HAB,AmPm",2018-04-06T15:56:26Z,https://i.scdn.co/image/ab67616d0000b2732b245d...,20
1100,Low - R3HAB Remix,"Greyson Chance,R3HAB",2018-04-06T15:56:21Z,https://i.scdn.co/image/ab67616d0000b273f0073a...,39
1101,Ain't That Why - Skytech Remix,"R3HAB,Krewella,Skytech",2018-04-06T15:56:19Z,https://i.scdn.co/image/ab67616d0000b2734c49a1...,49


In [42]:
df_saved_tracks[df_saved_tracks['popularity'] < 5]

,song_title,artists,date_added,popularity
39,Maze,"Mike Perry,Mangoo,Wanja Janeva",2021-11-09T11:38:20Z,2
96,Astronaut In The Ocean - Alok Remix,"Masked Wolf,Alok",2021-05-03T21:24:10Z,1
121,Hearts on Fire - Bassjackers Remix,"ILLENIUM,Dabin,Lights,Bassjackers",2021-02-12T20:50:37Z,0
152,Move IT,DJ Kuromi,2020-10-26T02:40:40Z,0
170,Back To You,Adam Rom,2020-10-15T03:23:46Z,0
...,...,...,...,...
1080,Wonderwall - Remastered,Oasis,2018-04-07T01:28:29Z,2
1081,Scream,"Tiësto,John Christian",2018-04-07T00:58:09Z,0
1083,Fade Into Darkness - Vocal Radio Mix,Avicii,2018-04-07T00:08:19Z,0
1086,Hold On,"Lane 8,Fractures",2018-04-06T20:47:36Z,0


In [64]:
songs['items'][1]['track']['album']['images'][0]['url']

'https://i.scdn.co/image/ab67616d0000b273d2f483a55b60cc858e2e8f4e'

In [57]:
for i in a:
    print(i)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/6cEuCEZu7PAE9ZSzLLc2oQ'}, 'href': 'https://api.spotify.com/v1/artists/6cEuCEZu7PAE9ZSzLLc2oQ', 'id': '6cEuCEZu7PAE9ZSzLLc2oQ', 'name': 'R3HAB', 'type': 'artist', 'uri': 'spotify:artist:6cEuCEZu7PAE9ZSzLLc2oQ'}
{'external_urls': {'spotify': 'https://open.spotify.com/artist/0Cd6nHYwecCNM1sVEXKlYr'}, 'href': 'https://api.spotify.com/v1/artists/0Cd6nHYwecCNM1sVEXKlYr', 'id': '0Cd6nHYwecCNM1sVEXKlYr', 'name': 'Krewella', 'type': 'artist', 'uri': 'spotify:artist:0Cd6nHYwecCNM1sVEXKlYr'}
{'external_urls': {'spotify': 'https://open.spotify.com/artist/4CrDEHL7ysNabeYvL3xjUX'}, 'href': 'https://api.spotify.com/v1/artists/4CrDEHL7ysNabeYvL3xjUX', 'id': '4CrDEHL7ysNabeYvL3xjUX', 'name': 'Skytech', 'type': 'artist', 'uri': 'spotify:artist:4CrDEHL7ysNabeYvL3xjUX'}
